In [1]:
import numpy as np
import matplotlib.pyplot as plt
import sounddevice as sd
import librosa
import soundfile as sf
import time
import librosa.display
from os import listdir, mkdir
from os.path import isfile, join, exists
from pydub import AudioSegment

c:\Users\gabriel\AppData\Local\Programs\Python\Python312\Lib\site-packages\pydub\utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)


## Fragmentando áudio

Para aumentar os dados foi feito:
- Injeção de barulho utilizando a função NumPy whiteNoise
- Time shifting (mover o áudio para direita ou esquerda)
- Velocidade

Extraindo características
- Zero-Crossing Rate
- RMSE
- MFCC

### TRABALHAR COM 1000 ÁUDIOS SAUDÁVEIS E 1000 ÁUDIOS PATOLÓGICOS


In [2]:
def create_dir(dir_path):
    try:
        print("Criando pasta: " + dir_path)
        mkdir(dir_path)
    except OSError as error:
        print(error)
        print() 

In [3]:
def path_to_save_func(audio_path, complemento, pasta = None):

    if pasta != None:
        path_pasta = audio_path + "/" + pasta
        if not exists(path_pasta):
            create_dir(path_pasta)

    splited = audio_path.split("/")
    file_name = splited[-1].replace(".wav", "")
    path_to_save = "/".join(splited[:len(splited) - 1]) + "/" + file_name + "_" + complemento + ".wav"
    print(path_to_save)
    return path_to_save

In [4]:
def inserir_white_noise(audio_path, pasta = None):

    # Carregar o áudio
    audio, sr = librosa.load(audio_path, sr=None)  # sr=None mantém a taxa de amostragem original

    # Criar o ruído branco
    mean = 0
    std_dev = 0.05  # Controle a intensidade do ruído
    white_noise = np.random.normal(loc=mean, scale=std_dev, size=len(audio))

    # Adicionando ruído ao sinal
    noisy_audio  = audio + white_noise

    # Normalizar para evitar saturação (opcional)
    noisy_audio = noisy_audio / np.max(np.abs(noisy_audio))

    # Reproduzindo o áudio
    sd.play(noisy_audio, samplerate=sr)
    sd.wait()  # Aguarda a reprodução terminar

    print("Áudio reproduzido!")

    # Salvar o áudio com ruído
    sf.write(path_to_save_func(audio_path, "white_noise", pasta), noisy_audio, sr)

    # Visualizando o sinal original e com ruído
    #plt.figure(figsize=(12, 6))

    #plt.subplot(2, 1, 1)
    #plt.plot(t[:1000], sine_wave[:1000])  # Mostrando apenas 1000 amostras
    #plt.title("Sinal Original")
    #plt.grid()

    #plt.subplot(2, 1, 2)
    #plt.plot(t[:1000], noisy_signal[:1000])
    #plt.title("Sinal com Ruído Branco")
    #plt.grid()

    #plt.tight_layout()
    #plt.show()



In [6]:
def time_strech(audio_path, pasta = None):

    # Carrega o áudio
    y, sr = librosa.load(audio_path)

    sd.play(y, samplerate=sr)

    # Aplica time-stretching
    rate = 1.5  # Ajuste o tempo: > 1 acelera, < 1 desacelera
    stretched_audio = librosa.effects.time_stretch(y, rate=rate)

    time.sleep(2)
    sd.play(stretched_audio, samplerate=sr)

    # Salva o áudio alterado
    sf.write(path_to_save_func(audio_path, "time_strech", pasta), stretched_audio, sr)

    # Exibe a forma de onda antes e depois (opcional)
    # plt.figure(figsize=(10, 4))
    # librosa.display.waveshow(y, sr=sr, alpha=0.5, label="Original")
    # librosa.display.waveshow(stretched_audio, sr=sr, color="r", alpha=0.5, label="Stretched")
    
    # plt.legend()
    # plt.show()

In [7]:
def time_shifting_with_silence(audio_path, pasta = None):

    # Carrega o áudio
    audio = AudioSegment.from_file(audio_path)

    # Define o deslocamento (em milissegundos)
    shift_ms = 500  # Desloca 500 ms

    # Adiciona silêncio no início ou no fim
    if shift_ms > 0:  # Desloca para frente
        shifted_audio = AudioSegment.silent(duration=shift_ms) + audio[:-shift_ms]
    else:  # Desloca para trás
        shift_ms = abs(shift_ms)
        shifted_audio = audio[shift_ms:] + AudioSegment.silent(duration=shift_ms)

    # Salva o áudio deslocado
    shifted_audio.export(path_to_save_func(audio_path, "time_shifting_with_silence", pasta), format="wav")



In [8]:
def time_shifting(audio_path, pasta = None, shift_direction = 1):
    if shift_direction != 1 and shift_direction != -1:
        raise Exception("Valores permitidos para shift_direction: 1 e -1")

    # Carrega o áudio
    y, sr = librosa.load(audio_path)
    sd.play(y, samplerate=sr)

    # Define o deslocamento (em número de amostras)
    shift = int(sr * 0.5)  # Desloca 0.5 segundos

    # Aplica o deslocamento
    shifted_audio = np.roll(y, shift * shift_direction)  # Desloca o áudio para frente

    time.sleep(2)
    sd.play(shifted_audio, samplerate=sr)

    # Para deslocar para trás, basta usar um valor negativo:
    # shifted_audio = np.roll(y, -shift)

    # Salva o áudio deslocado
    shift_mode = "front"
    if shift_direction == -1: 
        shift_mode = "back"
    sf.write(path_to_save_func(audio_path, "time_shifting" + "_" + shift_mode, pasta), shifted_audio, sr)


In [58]:
path_database = "D:/mestrado/separacao_svd_a_3_tons_aumentado/saudavel/masculino"

# masculino

# saudável

### Gerar 223 arquivos
count = 0

# Existem 773 arquivos
for arquivo in  listdir(path_database):
    if count > 223:
        break
    
    path_file = path_database + "/" + arquivo
    inserir_white_noise(path_file)
    count += 1
    

Áudio reproduzido!
D:/mestrado/separacao_svd_a_3_tons_aumentado/saudavel/masculino/694-saudavel_m_a_alto_white_noise.wav
Áudio reproduzido!
D:/mestrado/separacao_svd_a_3_tons_aumentado/saudavel/masculino/4-saudavel_m_a_neutro_white_noise.wav
Áudio reproduzido!
D:/mestrado/separacao_svd_a_3_tons_aumentado/saudavel/masculino/5-saudavel_m_a_neutro_white_noise.wav
Áudio reproduzido!
D:/mestrado/separacao_svd_a_3_tons_aumentado/saudavel/masculino/9-saudavel_m_a_neutro_white_noise.wav
Áudio reproduzido!
D:/mestrado/separacao_svd_a_3_tons_aumentado/saudavel/masculino/11-saudavel_m_a_neutro_white_noise.wav
Áudio reproduzido!
D:/mestrado/separacao_svd_a_3_tons_aumentado/saudavel/masculino/15-saudavel_m_a_neutro_white_noise.wav
Áudio reproduzido!
D:/mestrado/separacao_svd_a_3_tons_aumentado/saudavel/masculino/29-saudavel_m_a_neutro_white_noise.wav
Áudio reproduzido!
D:/mestrado/separacao_svd_a_3_tons_aumentado/saudavel/masculino/32-saudavel_m_a_neutro_white_noise.wav
Áudio reproduzido!
D:/mestra

In [67]:
path_database = "D:/mestrado/separacao_svd_a_3_tons_aumentado/polipo/masculino"

# masculino

# polipo: existem 51 itens

### Gerar 199 arquivos

count = 0
for arquivo in listdir(path_database):
    path_file = path_database + "/" + arquivo
    inserir_white_noise(path_file)
    time_strech(path_file)
    time_shifting(path_file, None, 1)
    time_shifting(path_file, None, -1)




Áudio reproduzido!
D:/mestrado/separacao_svd_a_3_tons_aumentado/polipo/masculino/109-polipo_m_a_alto_white_noise.wav
D:/mestrado/separacao_svd_a_3_tons_aumentado/polipo/masculino/109-polipo_m_a_alto_time_strech.wav
D:/mestrado/separacao_svd_a_3_tons_aumentado/polipo/masculino/109-polipo_m_a_alto_time_shifting_front.wav
D:/mestrado/separacao_svd_a_3_tons_aumentado/polipo/masculino/109-polipo_m_a_alto_time_shifting_back.wav
Áudio reproduzido!
D:/mestrado/separacao_svd_a_3_tons_aumentado/polipo/masculino/109-polipo_m_a_baixo_white_noise.wav
D:/mestrado/separacao_svd_a_3_tons_aumentado/polipo/masculino/109-polipo_m_a_baixo_time_strech.wav
D:/mestrado/separacao_svd_a_3_tons_aumentado/polipo/masculino/109-polipo_m_a_baixo_time_shifting_front.wav
D:/mestrado/separacao_svd_a_3_tons_aumentado/polipo/masculino/109-polipo_m_a_baixo_time_shifting_back.wav
Áudio reproduzido!
D:/mestrado/separacao_svd_a_3_tons_aumentado/polipo/masculino/109-polipo_m_a_neutro_white_noise.wav
D:/mestrado/separacao_svd

In [60]:
path_database = "D:/mestrado/separacao_svd_a_3_tons_aumentado/paralisia/masculino"

# masculino

# paralisia: existem 210 itens

### Gerar 40 arquivos
count = 0
for arquivo in  listdir(path_database):
    if count > 40:
        break
    
    path_file = path_database + "/" + arquivo
    inserir_white_noise(path_file)
    time_strech(path_file)
    count += 2


Áudio reproduzido!
D:/mestrado/separacao_svd_a_3_tons_aumentado/paralisia/masculino/128-paralisia_m_a_neutro_white_noise.wav
D:/mestrado/separacao_svd_a_3_tons_aumentado/paralisia/masculino/128-paralisia_m_a_neutro_time_strech.wav
Áudio reproduzido!
D:/mestrado/separacao_svd_a_3_tons_aumentado/paralisia/masculino/155-paralisia_m_a_neutro_white_noise.wav
D:/mestrado/separacao_svd_a_3_tons_aumentado/paralisia/masculino/155-paralisia_m_a_neutro_time_strech.wav
Áudio reproduzido!
D:/mestrado/separacao_svd_a_3_tons_aumentado/paralisia/masculino/358-paralisia_m_a_neutro_white_noise.wav
D:/mestrado/separacao_svd_a_3_tons_aumentado/paralisia/masculino/358-paralisia_m_a_neutro_time_strech.wav
Áudio reproduzido!
D:/mestrado/separacao_svd_a_3_tons_aumentado/paralisia/masculino/364-paralisia_m_a_neutro_white_noise.wav
D:/mestrado/separacao_svd_a_3_tons_aumentado/paralisia/masculino/364-paralisia_m_a_neutro_time_strech.wav
Áudio reproduzido!
D:/mestrado/separacao_svd_a_3_tons_aumentado/paralisia/ma

In [9]:
path_database = "D:/mestrado/separacao_svd_a_3_tons_aumentado/laringite/masculino"

# masculino

# laringite: tem 150 itens

### Gerar 100 arquivos

count = 0
for arquivo in  listdir(path_database):
    if count > 100:
        break
    
    path_file = path_database + "/" + arquivo
    inserir_white_noise(path_file)
    time_strech(path_file)
    time_shifting(path_file)
    count += 3

Áudio reproduzido!
D:/mestrado/separacao_svd_a_3_tons_aumentado/laringite/masculino/2605-laringite_m_a_neutro_white_noise.wav
D:/mestrado/separacao_svd_a_3_tons_aumentado/laringite/masculino/2605-laringite_m_a_neutro_time_strech.wav
D:/mestrado/separacao_svd_a_3_tons_aumentado/laringite/masculino/2605-laringite_m_a_neutro_time_shifting_front.wav
Áudio reproduzido!
D:/mestrado/separacao_svd_a_3_tons_aumentado/laringite/masculino/107-laringite_m_a_neutro_white_noise.wav
D:/mestrado/separacao_svd_a_3_tons_aumentado/laringite/masculino/107-laringite_m_a_neutro_time_strech.wav
D:/mestrado/separacao_svd_a_3_tons_aumentado/laringite/masculino/107-laringite_m_a_neutro_time_shifting_front.wav
Áudio reproduzido!
D:/mestrado/separacao_svd_a_3_tons_aumentado/laringite/masculino/139-laringite_m_a_neutro_white_noise.wav
D:/mestrado/separacao_svd_a_3_tons_aumentado/laringite/masculino/139-laringite_m_a_neutro_time_strech.wav
D:/mestrado/separacao_svd_a_3_tons_aumentado/laringite/masculino/139-laringi

In [62]:
path_database = "D:/mestrado/separacao_svd_a_3_tons_aumentado/disfonia/masculino"

# masculino

# disfonia: tem 87 itens

### Gerar 163 arquivos
count = 0
for arquivo in  listdir(path_database):
    if count > 163:
        break
    
    path_file = path_database + "/" + arquivo
    inserir_white_noise(path_file)
    time_strech(path_file)
    time_shifting(path_file)
    count += 3


Áudio reproduzido!
D:/mestrado/separacao_svd_a_3_tons_aumentado/disfonia/masculino/2356-disfonia_m_a_alto_white_noise.wav
D:/mestrado/separacao_svd_a_3_tons_aumentado/disfonia/masculino/2356-disfonia_m_a_alto_time_strech.wav
D:/mestrado/separacao_svd_a_3_tons_aumentado/disfonia/masculino/2356-disfonia_m_a_alto_time_shifting.wav
Áudio reproduzido!
D:/mestrado/separacao_svd_a_3_tons_aumentado/disfonia/masculino/2387-disfonia_m_a_alto_white_noise.wav
D:/mestrado/separacao_svd_a_3_tons_aumentado/disfonia/masculino/2387-disfonia_m_a_alto_time_strech.wav
D:/mestrado/separacao_svd_a_3_tons_aumentado/disfonia/masculino/2387-disfonia_m_a_alto_time_shifting.wav
Áudio reproduzido!
D:/mestrado/separacao_svd_a_3_tons_aumentado/disfonia/masculino/2436-disfonia_m_a_alto_white_noise.wav
D:/mestrado/separacao_svd_a_3_tons_aumentado/disfonia/masculino/2436-disfonia_m_a_alto_time_strech.wav
D:/mestrado/separacao_svd_a_3_tons_aumentado/disfonia/masculino/2436-disfonia_m_a_alto_time_shifting.wav
Áudio repr

---

In [ ]:
path_database = "D:/mestrado/separacao_svd_a_3_tons_aumentado/saudavel/feminino"

# feminino

# saudável tem 1284, não precisa gerar

In [12]:
path_database = "D:/mestrado/separacao_svd_a_3_tons_aumentado/polipo/feminino"

# feminino

# polipo: existem 30 itens

### Gerar 220 arquivos

count = 0
for arquivo in listdir(path_database):

    path_file = path_database + "/" + arquivo

    if "white_noise" not in arquivo:
        inserir_white_noise(path_file)
        count += 1
    if "time_strech" not in arquivo:
        time_strech(path_file)
        count += 1
    if "time_shifting" not in arquivo:
        time_shifting(path_file, None, 1)
        time_shifting(path_file, None, -1)
        count += 2




Áudio reproduzido!
D:/mestrado/separacao_svd_a_3_tons_aumentado/polipo/feminino/562-polipo_f_a_neutro_white_noise.wav
D:/mestrado/separacao_svd_a_3_tons_aumentado/polipo/feminino/562-polipo_f_a_neutro_time_strech.wav
D:/mestrado/separacao_svd_a_3_tons_aumentado/polipo/feminino/562-polipo_f_a_neutro_time_shifting_front.wav
D:/mestrado/separacao_svd_a_3_tons_aumentado/polipo/feminino/562-polipo_f_a_neutro_time_shifting_back.wav
Áudio reproduzido!
D:/mestrado/separacao_svd_a_3_tons_aumentado/polipo/feminino/932-polipo_f_a_neutro_white_noise.wav
D:/mestrado/separacao_svd_a_3_tons_aumentado/polipo/feminino/932-polipo_f_a_neutro_time_strech.wav
D:/mestrado/separacao_svd_a_3_tons_aumentado/polipo/feminino/932-polipo_f_a_neutro_time_shifting_front.wav
D:/mestrado/separacao_svd_a_3_tons_aumentado/polipo/feminino/932-polipo_f_a_neutro_time_shifting_back.wav
Áudio reproduzido!
D:/mestrado/separacao_svd_a_3_tons_aumentado/polipo/feminino/1052-polipo_f_a_neutro_white_noise.wav
D:/mestrado/separacao

In [ ]:
path_database = "D:/mestrado/separacao_svd_a_3_tons_aumentado/paralisia/feminino"

# feminino

# paralisia: tem 380, não precisa gerar

In [11]:
path_database = "D:/mestrado/separacao_svd_a_3_tons_aumentado/laringite/feminino"

# feminino

# laringite: tem 96 itens

### Gerar 154 arquivos

count = 0
for arquivo in  listdir(path_database):
    if count > 162:
        break
    
    path_file = path_database + "/" + arquivo
    inserir_white_noise(path_file)
    time_strech(path_file)
    time_shifting(path_file)
    count += 3

Áudio reproduzido!
D:/mestrado/separacao_svd_a_3_tons_aumentado/laringite/feminino/498-laringite_f_a_neutro_white_noise.wav
D:/mestrado/separacao_svd_a_3_tons_aumentado/laringite/feminino/498-laringite_f_a_neutro_time_strech.wav
D:/mestrado/separacao_svd_a_3_tons_aumentado/laringite/feminino/498-laringite_f_a_neutro_time_shifting_front.wav
Áudio reproduzido!
D:/mestrado/separacao_svd_a_3_tons_aumentado/laringite/feminino/568-laringite_f_a_neutro_white_noise.wav
D:/mestrado/separacao_svd_a_3_tons_aumentado/laringite/feminino/568-laringite_f_a_neutro_time_strech.wav
D:/mestrado/separacao_svd_a_3_tons_aumentado/laringite/feminino/568-laringite_f_a_neutro_time_shifting_front.wav
Áudio reproduzido!
D:/mestrado/separacao_svd_a_3_tons_aumentado/laringite/feminino/844-laringite_f_a_neutro_white_noise.wav
D:/mestrado/separacao_svd_a_3_tons_aumentado/laringite/feminino/844-laringite_f_a_neutro_time_strech.wav
D:/mestrado/separacao_svd_a_3_tons_aumentado/laringite/feminino/844-laringite_f_a_neutr

In [9]:
path_database = "D:/mestrado/separacao_svd_a_3_tons_aumentado/disfonia/feminino"

# feminino

# disfonia: tem 123 itens

### Gerar 127 arquivos
count = 0
for arquivo in  listdir(path_database):
    if count > 127:
        break
    
    path_file = path_database + "/" + arquivo
    inserir_white_noise(path_file)
    time_strech(path_file)
    time_shifting(path_file)
    count += 3


Áudio reproduzido!
D:/mestrado/separacao_svd_a_3_tons_aumentado/disfonia/feminino/2583-disfonia_f_a_neutro_white_noise.wav
D:/mestrado/separacao_svd_a_3_tons_aumentado/disfonia/feminino/2583-disfonia_f_a_neutro_time_strech.wav
D:/mestrado/separacao_svd_a_3_tons_aumentado/disfonia/feminino/2583-disfonia_f_a_neutro_time_shifting_front.wav
Áudio reproduzido!
D:/mestrado/separacao_svd_a_3_tons_aumentado/disfonia/feminino/2607-disfonia_f_a_neutro_white_noise.wav
D:/mestrado/separacao_svd_a_3_tons_aumentado/disfonia/feminino/2607-disfonia_f_a_neutro_time_strech.wav
D:/mestrado/separacao_svd_a_3_tons_aumentado/disfonia/feminino/2607-disfonia_f_a_neutro_time_shifting_front.wav
Áudio reproduzido!
D:/mestrado/separacao_svd_a_3_tons_aumentado/disfonia/feminino/368-disfonia_f_a_neutro_white_noise.wav
D:/mestrado/separacao_svd_a_3_tons_aumentado/disfonia/feminino/368-disfonia_f_a_neutro_time_strech.wav
D:/mestrado/separacao_svd_a_3_tons_aumentado/disfonia/feminino/368-disfonia_f_a_neutro_time_shift